# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [40]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

example_generator = square_root_generator(5)

print("Top 5:")
for sqrt_value in example_generator:
  print(sqrt_value)


-----
Top 5:
1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


Question 1 and 2:

In [39]:
# Sum usage:
print("-----")
sum_generator = square_root_generator(5)   
sum_top_items = sum(list(sum_generator));
print("Sum of top 5:", sum_top_items)

#Limit usage:
print("-----")
last_generator = square_root_generator(13)
last_generator_items = list(last_generator)
print("Top 13:", last_generator_items[-1])

-----
Sum of top 5: 8.382332347441762
-----
Top 13: 3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [46]:
%%capture
!pip install dlt[duckdb] 

In [47]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


Question 3:

In [48]:
import dlt

generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people_generators')


info = generators_pipeline.run(people_1(),
                                    table_name="people",
                                    write_disposition="replace")

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 2.29 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_generators
The duckdb destination used duckdb:///C:\Users\OTkachuk\Notebooks\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707867612.0612369 is LOADED and contains no failed jobs


In [49]:
import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

In [51]:
people = conn.sql("SELECT * FROM people").df()
display(people)

people_sum = conn.sql("SELECT SUM(age) FROM people").df()
display(people_sum)

,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1707867612.0612369,CpmBE8TZ4yck1Q
1,2,Person_2,27,City_A,1707867612.0612369,mnLzrqBf3E5xYg
2,3,Person_3,28,City_A,1707867612.0612369,cnzjyVXiLNJvdQ
3,4,Person_4,29,City_A,1707867612.0612369,TrN4Pr7VfBr41w
4,5,Person_5,30,City_A,1707867612.0612369,57i0a114ol+pjA


,sum(age)
0,140.0


In [57]:
info = generators_pipeline.run(people_2(),
                                    table_name="people",
                                    write_disposition="append")

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.62 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_generators
The duckdb destination used duckdb:///C:\Users\OTkachuk\Notebooks\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707867680.844431 is LOADED and contains no failed jobs


In [58]:
people = conn.sql("SELECT * FROM people").df()
display(people)

people_sum = conn.sql("SELECT SUM(age) FROM people").df()
display(people_sum)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707867612.0612369,CpmBE8TZ4yck1Q,None
1,2,Person_2,27,City_A,1707867612.0612369,mnLzrqBf3E5xYg,None
2,3,Person_3,28,City_A,1707867612.0612369,cnzjyVXiLNJvdQ,None
3,4,Person_4,29,City_A,1707867612.0612369,TrN4Pr7VfBr41w,None
4,5,Person_5,30,City_A,1707867612.0612369,57i0a114ol+pjA,None
5,3,Person_3,33,City_B,1707867680.844431,+L1Ni3gRqsqudg,Job_3
6,4,Person_4,34,City_B,1707867680.844431,fw1nkbHbm3ct8g,Job_4
7,5,Person_5,35,City_B,1707867680.844431,pOYcWWZ4YWEuZQ,Job_5
8,6,Person_6,36,City_B,1707867680.844431,oxpKPwzEC8/ynA,Job_6
9,7,Person_7,37,City_B,1707867680.844431,PYENbG4BbHYdsA,Job_7


,sum(age)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


Question 4:

In [62]:
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people_generators')


info = generators_pipeline.run(people_1(),
                                    table_name="people",
                                    write_disposition="replace",
                                    primary_key="id")

print(info)

info = generators_pipeline.run(people_2(),
                                    table_name="people",
                                    write_disposition="merge",
                                    primary_key="id")

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.40 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_generators
The duckdb destination used duckdb:///C:\Users\OTkachuk\Notebooks\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707868014.4605641 is LOADED and contains no failed jobs
Pipeline dlt_ipykernel_launcher load step completed in 0.69 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_generators
The duckdb destination used duckdb:///C:\Users\OTkachuk\Notebooks\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707868016.0146654 is LOADED and contains no failed jobs


In [63]:
people = conn.sql("SELECT * FROM people").df()
display(people)

people_sum = conn.sql("SELECT SUM(age) FROM people").df()
display(people_sum)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707868014.4605641,8NZrfORV+56DkA,None
1,2,Person_2,27,City_A,1707868014.4605641,BJEiuiUhLIZiHA,None
2,5,Person_5,35,City_B,1707868016.0146654,C3oeFvGmSw9iaQ,Job_5
3,7,Person_7,37,City_B,1707868016.0146654,CEJXDcA7Caqv1A,Job_7
4,4,Person_4,34,City_B,1707868016.0146654,4Xg4JPoJnrpiOA,Job_4
5,3,Person_3,33,City_B,1707868016.0146654,E4qReRMbLsDQHQ,Job_3
6,6,Person_6,36,City_B,1707868016.0146654,EM2azCH9gQv2xg,Job_6
7,8,Person_8,38,City_B,1707868016.0146654,xNpz7WD1u/jCug,Job_8


,sum(age)
0,266.0


# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [ ]:
# to do: homework :)

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX